# Thesis Defense Demo

#### Chaincode invoke command construction

In [26]:
# Common base for all invoke commands
def create_peer_invoke_command():
    # Contains the path to ca file, chaincode name and channel name
    base = "docker exec -it cli peer chaincode invoke -o orderer.organ.com:7050 --tls true --cafile /opt/gopath/src/github.com/hyperledger/fabric/peer/crypto/ordererOrganizations/organ.com/orderers/orderer.organ.com/msp/tlscacerts/tlsca.organ.com-cert.pem -C organ-channel -n organcc "
    return(base)

# Adding the peers to the chaincode invoke command
def add_peers(org='hl', peer_n=0):
    hl_p1_add = ' --peerAddresses peer1.histocompatibility.organ.com:7051 '
    hl_p1_cert = ' --tlsRootCertFiles /opt/gopath/src/github.com/hyperledger/fabric/peer/crypto/peerOrganizations/histocompatibility.organ.com/peers/peer1.histocompatibility.organ.com/tls/ca.crt '
    hl_p0_add = ' --peerAddresses peer0.histocompatibility.organ.com:7051 '
    hl_p0_cert = ' --tlsRootCertFiles /opt/gopath/src/github.com/hyperledger/fabric/peer/crypto/peerOrganizations/histocompatibility.organ.com/peers/peer0.histocompatibility.organ.com/tls/ca.crt '
    
    ho_p1_add = ' --peerAddresses peer1.hospital.organ.com:7051 '
    ho_p1_cert = ' --tlsRootCertFiles /opt/gopath/src/github.com/hyperledger/fabric/peer/crypto/peerOrganizations/hospital.organ.com/peers/peer1.hospital.organ.com/tls/ca.crt  '
    ho_p0_add = ' --peerAddresses peer0.hospital.organ.com:7051 '
    ho_p0_cert = ' --tlsRootCertFiles /opt/gopath/src/github.com/hyperledger/fabric/peer/crypto/peerOrganizations/hospital.organ.com/peers/peer0.hospital.organ.com/tls/ca.crt  '
    
    gp_p1_add = ' --peerAddresses peer1.gp.organ.com:7051 '
    gp_p1_cert = ' --tlsRootCertFiles /opt/gopath/src/github.com/hyperledger/fabric/peer/crypto/peerOrganizations/gp.organ.com/peers/peer1.gp.organ.com/tls/ca.crt  '
    gp_p0_add = ' --peerAddresses peer0.gp.organ.com:7051 '
    gp_p0_cert = ' --tlsRootCertFiles /opt/gopath/src/github.com/hyperledger/fabric/peer/crypto/peerOrganizations/gp.organ.com/peers/peer0.gp.organ.com/tls/ca.crt  '
    
    op_p1_add = ' --peerAddresses peer1.opo.organ.com:7051 '
    op_p1_cert = ' --tlsRootCertFiles /opt/gopath/src/github.com/hyperledger/fabric/peer/crypto/peerOrganizations/opo.organ.com/peers/peer1.opo.organ.com/tls/ca.crt  '
    op_p0_add = ' --peerAddresses peer0.opo.organ.com:7051 '
    op_p0_cert = ' --tlsRootCertFiles /opt/gopath/src/github.com/hyperledger/fabric/peer/crypto/peerOrganizations/opo.organ.com/peers/peer0.opo.organ.com/tls/ca.crt  '
    cmd = ""
    cmd += create_peer_invoke_command() + gp_p1_add + gp_p1_cert
    cmd += hl_p0_add + hl_p0_cert + hl_p1_add + hl_p1_cert
    cmd += ho_p0_add + ho_p0_cert + ho_p1_add + ho_p1_cert
    cmd += gp_p0_add + gp_p0_cert + gp_p1_add + gp_p1_cert
    cmd += op_p0_add + op_p0_cert + op_p1_add + op_p1_cert
    return(cmd)

# Add organ operation
def add_organ(organ_id = '', organ_name='', organ_data=''):
    cmd = add_peers() + '-c '
    
    cmd += '\'{"Args":["initOrgan", "' + str(organ_id) + '", "' + str(organ_name) + '", ' + str(organ_data) + ']}\''
    return(cmd)

#Read Organ operation
def read_organ(organ_id='421'):
    cmd = add_peers()
    cmd +=  '-c \'{\"Args\":[\"readOrgan",\"' + str(organ_id) + '\"]}\''
    return(cmd)

#Add Candidate operation
def add_candidate(candidate_id = '', organ_name='', info=''):
    cmd = add_peers() + '-c '
    cmd += '\'{"Args":["initCandidate", "' + candidate_id + '", "' + organ_name + '", ' + info + ']}\''
    return(cmd)

#Read Candidate operation
def read_candidate(candidate_id='421'):
    cmd = add_peers()
    cmd +=  '-c \'{\"Args\":[\"readCandidate",\"' + str(candidate_id) + '\"]}\''
    return(cmd)


### Save logs from running containers

In [27]:
def capture_logs():
    DATA_DIR = "/home/utsav_jain/demo_data"
    docker_container_list = []
    docker_container_list = !{"docker ps --format='{{ .Names}}'"}
    log_files = {}
    for container in docker_container_list:
        cmd = ("docker inspect %s --format={{'.LogPath'}}"%container)
        val = !{cmd}
        log_files[container] = val[0]
        copy_cmd = ("echo ' ' | sudo -S cp %s %s/%s.log" % (log_files[container], DATA_DIR ,container))
        !{copy_cmd}
        chomd = ("echo ' ' | sudo -S chmod 777 %s/%s.log" % (DATA_DIR, container))
        !{chomd}
    #     pwd = !{"pwd"}
        log_files[container] = ('%s/%s.log'%(DATA_DIR, container))


### Clean up environment for the next experiment

In [28]:
def docker_cleanup():
    !{ "docker rm -f $(docker ps -aq)"}
    !{ "docker volume prune -f"}
    !{ "docker network prune -f"}
    docker_images = "docker images --format '{{.Repository}}'"
    images = !{docker_images}
    dev_images=[]
    for image in images:
        if(image.find('dev-', 0, 6)>=0):
            dev_images.append(image)
    docker_rmi = "docker rmi -f "+" ".join(dev_images)
    print(docker_rmi)
    !{docker_rmi}

#### Reading Sample data

In [29]:
base = "/home/utsav_jain/RenewThesis/data"
with open(r"%s/data_1.json"%base, 'r') as file:
    data_1 = file.read()
print(type(data_1))
import json
data = []
data.append(json.dumps(data_1))
print((data_1))

<type 'str'>
{"NUM_KI_TX": 36695585, "PA_DISCARD_CD_OSTXT": "NpgFxoZCALvnNVZEZTFfxiBMPkZjhfHoOEFrUFjBvZKRhkiZPO", "CARDIAC_OUTPUT_CATH_POST_DON": 22930718, "HIST_OTH_DRUG_DON": "z", "KIL_REASON_OSTXT": "idtYtAFvCdftglvtwLhriYyWxceRySHQrrBrxUAwuImDQQKRQY", "INS2_DISPOSITION": 15268362, "INS1_REASON_CD": 99311362, "RT_KI_PUMP_TY": 9750260, "LIS2_BACK_TBL_FLUSH": 78330444, "PCWP_INIT_DON": 12111900, "HYPERTENS_DUR_DON": 60974435, "REGION": 21239455, "LUB_DISCARD_CD": 63867131, "LUR_DISCARD_CD_OSTXT": "AOdUKZrfcFZlPwkAviFUjQcVDEoYNhICOkiZUHEFidCLqNFSZR", "CVP_CATH_POST_DON": 21466655, "ETHNICITY_DON": 61200277, "IN_FINAL_FLUSH": 59564084, "PT_OTH1_OSTXT_DON": "hqpqWEwlBhmZaGRliPuHiJarslSfDZHdWDPYEnfhmjTAddjqFwfGAWThpzEvxsHDhxBEFjidIpx", "HBV_DNA": "bW", "PULM_INF_DON": 82167164, "INS2_INITIAL_FLUSH": 86481245, "PT_OTH2_OSTXT_DON": "RdCbGCtuheBtiHevGGLYSIgBgbwBfblgeKPYWGltbTSBRuOCiNRkkvNuGHmJbcaoYxwSJZPWuEg", "LI_REASON_OSTXT": "RzwfgkDpqLUrAfNpAmLJeDBLuyPcfRgglBKgMxGFkWofMDpnDn", "INS1_BAC

#### Printing sample command 

In [30]:
cmd = add_organ(str(8001), "intestine", "Organ Data")
print(cmd)

docker exec -it cli peer chaincode invoke -o orderer.organ.com:7050 --tls true --cafile /opt/gopath/src/github.com/hyperledger/fabric/peer/crypto/ordererOrganizations/organ.com/orderers/orderer.organ.com/msp/tlscacerts/tlsca.organ.com-cert.pem -C organ-channel -n organcc  --peerAddresses peer1.gp.organ.com:7051  --tlsRootCertFiles /opt/gopath/src/github.com/hyperledger/fabric/peer/crypto/peerOrganizations/gp.organ.com/peers/peer1.gp.organ.com/tls/ca.crt   --peerAddresses peer0.histocompatibility.organ.com:7051  --tlsRootCertFiles /opt/gopath/src/github.com/hyperledger/fabric/peer/crypto/peerOrganizations/histocompatibility.organ.com/peers/peer0.histocompatibility.organ.com/tls/ca.crt  --peerAddresses peer1.histocompatibility.organ.com:7051  --tlsRootCertFiles /opt/gopath/src/github.com/hyperledger/fabric/peer/crypto/peerOrganizations/histocompatibility.organ.com/peers/peer1.histocompatibility.organ.com/tls/ca.crt  --peerAddresses peer0.hospital.organ.com:7051  --tlsRootCertFiles /opt/g

#### Adding An organ

In [31]:
cmd = add_organ(str(8002), "intestine", data[0])
print(cmd)
! {cmd}

docker exec -it cli peer chaincode invoke -o orderer.organ.com:7050 --tls true --cafile /opt/gopath/src/github.com/hyperledger/fabric/peer/crypto/ordererOrganizations/organ.com/orderers/orderer.organ.com/msp/tlscacerts/tlsca.organ.com-cert.pem -C organ-channel -n organcc  --peerAddresses peer1.gp.organ.com:7051  --tlsRootCertFiles /opt/gopath/src/github.com/hyperledger/fabric/peer/crypto/peerOrganizations/gp.organ.com/peers/peer1.gp.organ.com/tls/ca.crt   --peerAddresses peer0.histocompatibility.organ.com:7051  --tlsRootCertFiles /opt/gopath/src/github.com/hyperledger/fabric/peer/crypto/peerOrganizations/histocompatibility.organ.com/peers/peer0.histocompatibility.organ.com/tls/ca.crt  --peerAddresses peer1.histocompatibility.organ.com:7051  --tlsRootCertFiles /opt/gopath/src/github.com/hyperledger/fabric/peer/crypto/peerOrganizations/histocompatibility.organ.com/peers/peer1.histocompatibility.organ.com/tls/ca.crt  --peerAddresses peer0.hospital.organ.com:7051  --tlsRootCertFiles /opt/g

2019-10-28 14:55:22.270 UTC [chaincodeCmd] chaincodeInvokeOrQuery -> INFO 001 Chaincode invoke successful. result: status:200 payload:"\016\211\336\"x\255:\270\032" 


#### Reading that organ

In [32]:
cmd = read_organ(8002)
print(cmd)
!{cmd}

docker exec -it cli peer chaincode invoke -o orderer.organ.com:7050 --tls true --cafile /opt/gopath/src/github.com/hyperledger/fabric/peer/crypto/ordererOrganizations/organ.com/orderers/orderer.organ.com/msp/tlscacerts/tlsca.organ.com-cert.pem -C organ-channel -n organcc  --peerAddresses peer1.gp.organ.com:7051  --tlsRootCertFiles /opt/gopath/src/github.com/hyperledger/fabric/peer/crypto/peerOrganizations/gp.organ.com/peers/peer1.gp.organ.com/tls/ca.crt   --peerAddresses peer0.histocompatibility.organ.com:7051  --tlsRootCertFiles /opt/gopath/src/github.com/hyperledger/fabric/peer/crypto/peerOrganizations/histocompatibility.organ.com/peers/peer0.histocompatibility.organ.com/tls/ca.crt  --peerAddresses peer1.histocompatibility.organ.com:7051  --tlsRootCertFiles /opt/gopath/src/github.com/hyperledger/fabric/peer/crypto/peerOrganizations/histocompatibility.organ.com/peers/peer1.histocompatibility.organ.com/tls/ca.crt  --peerAddresses peer0.hospital.organ.com:7051  --tlsRootCertFiles /opt/g

#### Saving Logs

In [33]:
capture_logs()

#### Cleaning up environment

In [25]:
docker_cleanup()

9780edc0e9c8
3fe8e6238eb7
8ccfd5474eba
3de8fd10c789
6324ea3a993b
7105ea5067ee
ee5726c3d20d
e55ccec20709
a76b5b6132d3
4e4153337ed0
4389e507f28d
76820373fc5b
282da80fd242
38dd53c2ff93
bf666b58e762
92cf7a5bd50f
ad59ab614127
1051aa1e86e3
Deleted Volumes:
experiment4transactionrate_peer0.hospital.organ.com
experiment4transactionrate_peer0.opo.organ.com
experiment4transactionrate_peer1.hospital.organ.com
experiment4transactionrate_peer0.gp.organ.com
experiment4transactionrate_peer1.opo.organ.com
experiment4transactionrate_peer0.histocompatibility.organ.com
experiment4transactionrate_peer1.gp.organ.com
f903f8a26f8ee0b0152284758c9f3bf9aff523fbc23901daaef9219a39304c9d
experiment4transactionrate_peer1.histocompatibility.organ.com
experiment4transactionrate_orderer.organ.com

Total reclaimed space: 2.598MB
Deleted Networks:
topo2_organ_chain_network
experiment4transactionrate_organ_chain_network

docker rmi -f dev-peer1.opo.organ.com-organcc-1.0-f7ab76561c21a418dff95bfcdf78a45d8518b5f2d2be784ba3c

In [34]:
example_log = "grep Received /home/utsav_jain/demo_data/peer0.gp.organ.com.log"
! {example_log}

{"log":"\u001b2019-10-28 14:52:42.718 UTC [gossip.privdata] StoreBlock -\u003e INFO 050\u001b [organ-channel] Received block [1] from buffer\n","stream":"stderr","time":"2019-10-28T14:52:42.719127547Z"}
{"log":"\u001b2019-10-28 14:52:42.746 UTC [gossip.privdata] StoreBlock -\u003e INFO 059\u001b [organ-channel] Received block [2] from buffer\n","stream":"stderr","time":"2019-10-28T14:52:42.747045296Z"}
{"log":"\u001b2019-10-28 14:52:42.777 UTC [gossip.privdata] StoreBlock -\u003e INFO 066\u001b [organ-channel] Received block [3] from buffer\n","stream":"stderr","time":"2019-10-28T14:52:42.777181161Z"}
{"log":"\u001b2019-10-28 14:52:42.813 UTC [gossip.privdata] StoreBlock -\u003e INFO 073\u001b [organ-channel] Received block [4] from buffer\n","stream":"stderr","time":"2019-10-28T14:52:42.813848974Z"}
{"log":"\u001b2019-10-28 14:53:14.415 UTC [gossip.privdata] StoreBlock -\u003e INFO 08b\u001b [organ-channel] Received block [5] from buffer\n","stream":"stderr","time":"2019-10-28T14:53:1